### Exercise1
Write a program to select columns from a DataFrame with more than half values are not NA.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6],
                   [None, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6
3,NaN,4.0,6


In [3]:
df.dropna(axis=1, thresh=df.shape[1]/2+1)

,1,2
0,NaN,2
1,3.0,5
2,4.0,6
3,4.0,6


### Exercise 2 
Fill the NA values in a DataFrame with the average value of the row that NA belongs to.

In [4]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6],
                   [None, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6
3,NaN,4.0,6


In [5]:
# Hint: the fillna can take a list as input and fill NA in each column using the correspondance value in the list
# You can make use of transpose operation to fill NA by rows
avg_list = df.mean(axis=1)
df.T.fillna(avg_list).T

,0,1,2
0,1.0,1.5,2.0
1,2.0,3.0,5.0
2,5.0,4.0,6.0
3,5.0,4.0,6.0


Similar to the lecture note, we'll define the following two functions which creates a ``DataFrame`` of a particular form and display multiple ``DataFrame``s side by side:

In [6]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        # str.join(sequence): use str to join the given sequence
        # eval(expression): return the result of the expression
        # dataframe object has _repr_html_ method
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

### Exercise 3
Write a program to concatenate two DataFrames by the clumns 'B'  of first DataFrame and providing the labels for data sources, i.e., 'first' and 'second'. Then select values with row index 1 on second level.


In [7]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [1, 2])
print('Dataframes before concatenation:')
display('df5', 'df6')

Dataframes before concatenation:


,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
1,B1,C1,D1
2,B2,C2,D2


In [8]:
print('Dataframe after concatenation:')
indx = ['B']
colume=pd.Series(df5.columns)
colume_for_indx = [x for x in colume.index if colume[x] in indx]
df_con = pd.concat([df5, df6],  join_axes=[df5.columns[colume_for_indx]], keys=['first','second'], sort=True)
df_con

Dataframe after concatenation:


B
first  1  B1
       2  B2
second 1  B1
       2  B2

In [9]:
print('Values after selection:')
idx = pd.IndexSlice
df_sel = df_con.loc[idx[:,1],:]
df_sel

Values after selection:


,,B
first,1,B1
second,1,B1


### Exercise 4

Consider the following data, which contains 10 rows of ages, year and population information. Write a program to calculate the max population of each year and age. The output results should be display in a two-dimentional array style, with rows indicating the year and columns indicating ages. 

In [10]:
population = pd.DataFrame({'ages':['under18','under18','total','under18','total', 'total', 'under18','total','under18','total'],
              'year':[2012,2010,2010,2010,2012,2012,2010,2012,2012,2012],
             'population':[11174,48175,98723,np.nan,11257,1140,23654,np.nan,1637,12653]}
            )
population

,ages,year,population
0,under18,2012,11174.0
1,under18,2010,48175.0
2,total,2010,98723.0
3,under18,2010,NaN
4,total,2012,11257.0
5,total,2012,1140.0
6,under18,2010,23654.0
7,total,2012,NaN
8,under18,2012,1637.0
9,total,2012,12653.0


In [16]:
population.groupby(['year','ages']).aggregate({
                             'population': max}).unstack()

population         
ages      total  under18
year                    
2010    98723.0  48175.0
2012    12653.0  11174.0

### Exercise 5
Consider the data used in Exercise 4. The population column has some missing values. Write a program to replace the missing value by the average population of the same year and same type. For example, the population of third row should be replaced by the average population of year '2010' and ages 'under18'.

In [12]:
population = pd.DataFrame({'ages':['under18','under18','total','under18','total', 'total', 'under18','total','under18','total'],
              'year':[2012,2010,2010,2010,2012,2012,2010,2012,2012,2012],
             'population':[11174,48175,98723,np.nan,11257,1140,23654,np.nan,1637,12653]}
            )
population

,ages,year,population
0,under18,2012,11174.0
1,under18,2010,48175.0
2,total,2010,98723.0
3,under18,2010,NaN
4,total,2012,11257.0
5,total,2012,1140.0
6,under18,2010,23654.0
7,total,2012,NaN
8,under18,2012,1637.0
9,total,2012,12653.0


In [15]:
def missing_pop(population):
    """Solution 1"""
    n_population = population[population['population'].isnull()]
    for indexs in n_population.index:
        year = population.loc[indexs,'year']
        age = population.loc[indexs,'ages']
        pop_year = population[population['year']==year]
        pop_type_year = pop_year[pop_year['ages']== age]
#         print(pop_type_year)
        mean = pop_type_year['population'].dropna().mean()
        population.loc[indexs,'population'] = int(mean)
        
        
missing_pop(population)

print("Dataframe after the missing value being replaced:")
population

Dataframe after the missing value being replaced:


,ages,year,population
0,under18,2012,11174.0
1,under18,2010,48175.0
2,total,2010,98723.0
3,under18,2010,35914.0
4,total,2012,11257.0
5,total,2012,1140.0
6,under18,2010,23654.0
7,total,2012,8350.0
8,under18,2012,1637.0
9,total,2012,12653.0


In [14]:
def missing_pop1(population):
    """Solution 2 using groupby"""
    mean_data = population.groupby(['year','ages'])['population'].mean()
    n_population = population[population['population'].isnull()]
    for indexs in n_population.index:
        year = population.loc[indexs,'year']
        age = population.loc[indexs,'ages']
        population.loc[indexs,'population'] = mean_data.loc[year,age]
        
missing_pop1(population)
population

,ages,year,population
0,under18,2012,11174.0
1,under18,2010,48175.0
2,total,2010,98723.0
3,under18,2010,35914.0
4,total,2012,11257.0
5,total,2012,1140.0
6,under18,2010,23654.0
7,total,2012,8350.0
8,under18,2012,1637.0
9,total,2012,12653.0
